<a href="https://colab.research.google.com/github/aswini03Aswini/Unhack/blob/main/Milestone1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install opencv-python-headless pandas

In [6]:
import zipfile
import os

# Unzip the dataset
zip_path = '/content/DataSet.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/dataset')

# List the files
dataset_path = '/content/dataset'
print(os.listdir(dataset_path))


['DataSet']


In [9]:
import cv2
import pandas as pd
import os

def load_templates(template_folder):
    templates = {}
    for filename in os.listdir(template_folder):
        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".PNG"):
            template_path = os.path.join(template_folder, filename)
            templates[filename] = cv2.imread(template_path, 0)
    return templates

def match_template(image, templates):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    detected_speed = None
    max_val = 0

    for template_name, template in templates.items():
        result = cv2.matchTemplate(gray_image, template,cv2.TM_CCOEFF_NORMED)
        _, max_val_curr, _, _ = cv2.minMaxLoc(result)

        if max_val_curr > max_val:
            max_val = max_val_curr
            detected_speed = template_name

    if detected_speed:
        return detected_speed.split('.')[0]  # Assuming the template filenames are the speed limits (e.g., '30.jpg')
    return None

def process_images(input_folder, template_folder, output_csv):
    templates = load_templates(template_folder)
    results = []

    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            speed_limit = match_template(image, templates)
            results.append([filename, speed_limit[9:]])

    df = pd.DataFrame(results, columns=["Input image name", "Speed limit"])
    df.to_csv(output_csv, index=False)

# Define your paths

input_folder='/content/dataset/DataSet/Milestone 1'
template_folder='/content/dataset/DataSet/Template images'
output_csv='M1_Output.csv'

# Run the process
process_images(input_folder, template_folder, output_csv)

